# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [16]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yangambi,-25.3,45.4833,38.0,28,1,1.44,MG,1731921055
1,1,revda,-25.3,45.4833,38.0,28,1,1.44,MG,1731921055
2,2,posto fiscal rolim de moura,-25.3,45.4833,38.0,28,1,1.44,MG,1731921055
3,3,waitangi,-25.3,45.4833,38.0,28,1,1.44,MG,1731921055
4,4,blackmans bay,-25.3,45.4833,38.0,28,1,1.44,MG,1731921055


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [18]:
city_map = city_data_df.hvplot.points(
    x="Lng",                # Use the 'Lng' column for longitude (x-axis)
    y="Lat",                # Use the 'Lat' column for latitude (y-axis)
    geo=True,               # Enable geographic plotting
    size="Humidity",        # Size of the point based on 'Humidity' column
    scale=1,                # Adjust scale of point sizes if necessary
    color="City",           # Color points by city name or another category
    alpha=0.6,              # Set transparency for better visualization
    tiles="OSM",            # Use OpenStreetMap for background
    frame_width=800,        # Set the width of the map
    frame_height=600        # Set the height of the map
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) &  # Temperature condition
    (city_data_df["Wind Speed"] < 4.5) &                                # Wind speed condition
    (city_data_df["Cloudiness"] == 0)                                   # Cloudiness condition
]

# Display the filtered DataFrame
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Create a new DataFrame with selected columns for the hotel search
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add a new column "Hotel Name"
hotel_df["Hotel Name"] = ""

# Display a preview of the DataFrame
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Define parameters for searching hotels
radius = 1000  # Search radius in meters
params = {
    "categories": "accommodation.hotel",  # Category to search for hotels
    "apiKey": geoapify_key,              # Geoapify API key
    "limit": 20                          # Maximum number of results to retrieve
}

# Notify the user that the hotel search is starting
print("Starting hotel search...")

# Loop through each row in the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Extract latitude and longitude for the current city
    lat = row["Lat"]
    lon = row["Lng"]

    # Update the parameters with the location filter and bias
    params["filter"] = f"circle:{lon},{lat},{radius}"  # Define search area
    params["bias"] = f"proximity:{lon},{lat}"         # Bias results towards the city's location

    # Define the base URL for the Geoapify Places API
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request with the defined parameters
    response = requests.get(base_url, params=params)

    # Parse the API response to JSON format
    response_json = response.json()
    
    # Attempt to retrieve the first hotel's name from the API results
    try:
        hotel_name = response_json["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name  # Update the DataFrame with the hotel name
    except (KeyError, IndexError):
        # If no hotel is found, set the name to "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the result for the current city
    print(f"{row['City']} - Nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display a preview of the updated DataFrame
hotel_df

Starting hotel search...


,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x="Lng",                 # Longitude for x-axis
    y="Lat",                 # Latitude for y-axis
    geo=True,                # Enable geographic plotting
    size="Humidity",         # Point size based on humidity
    scale=1,                 # Scale for point size
    color="City",            # Color points by city
    alpha=0.5,               # Set transparency for better visualization
    tiles="OSM",             # Use OpenStreetMap tiles as the base layer
    frame_width=700,         # Set map width
    frame_height=500,        # Set map height
    hover_cols=["Hotel Name", "Country"]  # Add hotel name and country in hover tooltips
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)